In [4]:
!pip install torchdata #iopath

  Using cached torchdata-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [5]:
import sys

print(sys.version)

3.10.6 (main, Oct  7 2022, 20:19:58) [GCC 11.2.0]


In [30]:
!aws s3 ls --recursive s3://dsoaws/bloom/output/model_from_notebook_ray/

In [6]:
!mkdir -p /etc/pki/tls/certs && cp /etc/ssl/certs/ca-certificates.crt /etc/pki/tls/certs/ca-bundle.crt


In [7]:
import torchdata
from torchdata.datapipes.iter import IterableWrapper
 
S3_BUCKET = "s3://dsoaws/data/train/part-algo-1-womens_clothing_ecommerce_reviews.csv"
    
train_files = IterableWrapper([S3_BUCKET]).list_files_by_s3()
train_files

S3FileListerIterDataPipe

In [8]:
from torchdata.datapipes.iter import IoPathFileOpener

train_files = train_files.shuffle().cycle()

print(next(iter(train_files)))

s3://dsoaws/data/train/part-algo-1-womens_clothing_ecommerce_reviews.csv


In [9]:
from torchdata.datapipes.iter import S3FileLoader, IterableWrapper

train_file_loader = S3FileLoader(train_files)

print(next(iter(train_file_loader)))

#     samples = tars.groupby(lambda x:     
#                                os.path.basename(x[0]).split(".")[0],
#                            group_size=2, guaranteed_group_size=2)
#     dataset = samples.map(lambda x: 
#                   {'image': x[0][1].read(),
#                    'label': x[0][1].read()})
#     dataset = dataset.shuffle(buffer_size=10)

('s3://dsoaws/data/train/part-algo-1-womens_clothing_ecommerce_reviews.csv', StreamWrapper<<_io.BytesIO object at 0x7f8bfd267d30>>)


In [10]:
# ERROR
# ProfileNotFound: The config profile (saml) could not be found
# This exception is thrown by __iter__ of IoPathFileOpenerIterDataPipe(mode='rb:gz', pathmgr=<iopath.common.file_io.PathManager object at 0x7f93429e7760>, source_datapipe=CyclerIterDataPipe)

# tars = IoPathFileOpener(urls, mode="rb:gz").load_from_tar()

# print(next(iter(tars)))

In [15]:
dataset = train_file_loader.map(lambda x: 
              {'image': x[0][1],
               'label': x[0][1]})

In [18]:
import os
samples = dataset.groupby(lambda x: os.path.basename(x[0]).split(".")[0],
                       group_size=2, guaranteed_group_size=2)

samples

GrouperIterDataPipe

In [21]:
print(next(iter(samples)))

KeyError: '\nThis exception is thrown by __iter__ of GrouperIterDataPipe(datapipe=MapperIterDataPipe, drop_remaining=False, group_key_fn=<lambda>, group_size=2, guaranteed_group_size=2)'

In [26]:
import datasets

dataset = samples.map(datasets.parse_csv(delimiter='\t'))

# .map(lambda x: 
#               {'customer_id': x[0][1].read(),
#                'product_id': x[0][1].read()})
dataset = dataset.shuffle(buffer_size=10)
dataset

print(next(iter(dataset)))

AttributeError: module 'datasets' has no attribute 'parse_csv'

In [ ]:
# from torchdata.datapipes.iter import S3FileLoader, IterableWrapper
# def get_dataset():
#     urls = [f's3://<path in s3>/{i}.tar' for i in range(num_files)]
#     urls = IterableWrapper(urls).shuffle().cycle()
#     tars = S3FileLoader(urls).load_from_tar()
#     samples = tars.groupby(lambda x:     
#                                os.path.basename(x[0]).split(".")[0],
#                            group_size=2, guaranteed_group_size=2)
#     dataset = samples.map(lambda x: 
#                   {'image':dataset = samples.map(lambda x: 
#               {'customer_id': x[0][1].read(),
#                'product_id': x[0][1].read()})
# dataset = dataset.shuffle(buffer_size=10)
# dataset x[0][1].read(),
#                    'label': x[0][1].read()})
#     dataset = dataset.shuffle(buffer_size=10)
# return dataset

In [23]:
datapipe_s3 = datapipe_s3.parse_csv(delimiter='\t')

NameError: name 'datapipe_s3' is not defined

In [ ]:
N_ROWS = 100  # total number of rows of data
train, validation = datapipe_s3.random_split(total_length=N_ROWS, weights={"train": 0.8, "valid": 0.2}, seed=42)

# type(train)
# type(validation)

for x in train:  # Iterating through the training dataset
    print(x)

for y in validation:  # Iterating through the validation dataset
    print(y)